In [ ]:
#| hide
from Single_Cell_Fuzzy_Labels.core import *

# Single-Cell-Fuzzy-Labels

> This GitHub repo offers a method for label transfer in single-cell RNA-seq data using shared embeddings from foundation models. It leverages language model APIs (GPT-3.5 or 4) for harmonizing label sets, enhancing transfer interpretation and facilitating basic metric evaluation.

This file will become your README and also the index of your documentation.

## Install

```sh
pip install Single_Cell_Fuzzy_Labels
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2